In [ ]:
#Exo 8
# Exercise: Load the MNIST data and split it into a training set, a validation set, and a test set (e.g., use 50,000 instances for training, 10,000 for validation, and 10,000 for testing).
# train multiple classifiers
# stack them into a voting classifier soft or hard and get the best result

In [3]:
from sklearn.datasets import fetch_openml

X_mnist, y_mnist =  fetch_openml("mnist_784", return_X_y=True, as_frame=False, parser='auto')

In [4]:
X_train, y_train = X_mnist[:50000], y_mnist[:50000]
X_valid, y_valid = X_mnist[50000:600000], y_mnist[50000:600000]
X_test, y_test = X_mnist[60000:], y_mnist[60000:]

In [7]:
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score

random_forest_clf = RandomForestClassifier(random_state=42)
extra_trees_clf = ExtraTreesClassifier(random_state=42)
svm_clf = LinearSVC(dual=True, random_state=42, learning_rate_init=0.005)

my_army = [random_forest_clf, extra_trees_clf, svm_clf]

for soldier in my_army:
    print(soldier, "in training")
    soldier.fit(X_train,y_train)

RandomForestClassifier(random_state=42) in training
ExtraTreesClassifier(random_state=42) in training
LinearSVC(dual=True, random_state=42) in training


c:\Users\ahmed\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [9]:
[estimator.score(X_valid, y_valid) for estimator in my_army] #damn svc is dumb as hell or takes ages to train

[0.9708, 0.9723, 0.8588]

In [13]:
from sklearn.ensemble import VotingClassifier

my_army = [
    ("rf", random_forest_clf),
    ("et", extra_trees_clf),
    ("svm", svm_clf)
]

full_army = VotingClassifier(my_army, voting="hard", n_jobs=-1)
soft_army = VotingClassifier(my_army, voting="soft", n_jobs=-1)

In [14]:
full_army.fit(X_train,y_train)
soft_army.fit(X_train,y_train)

VotingClassifier(estimators=[('rf', RandomForestClassifier(random_state=42)),
                             ('et', ExtraTreesClassifier(random_state=42)),
                             ('svm', LinearSVC(dual=True, random_state=42))],
                 n_jobs=-1, voting='soft')

In [15]:
print("the hard boiled guys got: ", full_army.score(X_valid, y_valid))
print("the soft twink guys: ", soft_army.score(X_valid, y_valid)) # LinearSVC doesnt support proba calculation so im not doing it i got 0.97 and its good enough

the hard boiled guys got:  0.9707


AttributeError: 'LinearSVC' object has no attribute 'predict_proba'

In [ ]:
#Exo 9
# Exercise: Run the individual classifiers from the previous exercise to make predictions on the validation set, 
# and create a new training set with the resulting predictions: each training instance is a vector containing the set of predictions from all your classifiers for an image,
# and the target is the image's class. Train a classifier on this new training set.

In [20]:
#collecting the predictions
import numpy as np

def get_model_predictions(models, X_pred):
    predictions = []

    for name, model in models:
        print(f"Predicting with {name}...")
        preds = model.predict(X_pred)

        predictions.append(preds)

    # stack into shape (n_samples, n_models)
    pred_matrix = np.column_stack(predictions)
    return pred_matrix

predictions = get_model_predictions(my_army, X_valid)

Predicting with rf...
Predicting with et...
Predicting with svm...


In [21]:
from sklearn.ensemble import RandomForestClassifier

blender = RandomForestClassifier(random_state=67, oob_score=True)

blender.fit(predictions,y_valid)

blender.oob_score_

0.9722

In [22]:
#testing the result of the blender on the est
from sklearn.metrics import accuracy_score

accuracy_score(y_test, blender.predict(get_model_predictions(my_army, X_test)))

Predicting with rf...
Predicting with et...
Predicting with svm...


0.9757

In [24]:
#trying with the stacking classifier because the book said so ig
from sklearn.ensemble import StackingClassifier

stack_clf = StackingClassifier(my_army,
                               final_estimator=blender)

X_train_full = X_mnist[:60000]
y_train_full = y_mnist[:60000]
stack_clf.fit(X_train_full, y_train_full)

c:\Users\ahmed\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\ahmed\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\ahmed\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\ahmed\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\ahmed\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warni

StackingClassifier(estimators=[('rf', RandomForestClassifier(random_state=42)),
                               ('et', ExtraTreesClassifier(random_state=42)),
                               ('svm', LinearSVC(dual=True, random_state=42))],
                   final_estimator=RandomForestClassifier(oob_score=True,
                                                          random_state=67))

In [25]:
stack_clf.score(X_test,y_test)

0.9766